# Setup

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
cd /content/drive/MyDrive/Thesis_Organized/Detectron2/Inference_testset/

/content/drive/MyDrive/Thesis_Organized/Detectron2/Inference_testset


In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [2]:
import os
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

In [3]:
Data_cfg = {

  # COCO TRAINING DATASET PATH
  "Coco_labels_train_dir": "/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_train_data.json",

  # COCO TESTING DATASET PATH
  "Coco_labels_test_dir": "/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_test_data.json",

  # COCO TRAINING VALIDATION PATH
  "Coco_labels_val_dir": "/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json",

  # CROPPED IMAGES PATH
  "cropped_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Images_cropped",

  # ORIGINAL IMAGES PATH
  "original_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Original Images",

  # GROUND TRUTH IMAGES PATH
  "GT_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Ground Truth",

  # PATH TO THE CSV FILES OF THE ANNOTATIONS CREATED IN THE DATA PROCESSING PART
  "annotations_path": "/content/drive/MyDrive/Thesis_Organized/Data/Ground Truth/annotations.csv"

}

In [4]:
import pandas as pd

data_path = Data_cfg["annotations_path"]
df = pd.read_csv(data_path).drop(columns = ["Unnamed: 0"])
df.head()

,id,name,track,z_order,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,folder,bbox_x,bbox_y,bbox_width,bbox_height,segmentation
0,0,L1Ap5_1-1-109.png,Track1,0,411.65,248.47,312.63,149.45,223.32,36.45,Train,209.48,254.64,217.19,239.21,"[217.19, 27.01, 215.91, 28.29, 214.62, 29.58, ..."
1,0,L1Ap5_1-1-109.png,Track2,0,384.92,256.98,339.97,263.66,406.79,249.08,Train,327.71,237.92,93.82,43.73,"[399.68, 239.21, 398.4, 240.5, 397.11, 240.5, ..."
2,1,L1Ap5_1-1-142.png,Track1,0,567.18,283.10,495.49,275.20,440.15,264.27,Val,427.96,218.63,151.65,48.87,"[435.67, 253.36, 434.38, 254.64, 433.1, 254.64..."
3,1,L1Ap5_1-1-142.png,Track2,0,316.88,223.56,243.37,181.04,130.37,128.18,Val,116.95,279.08,214.62,124.75,"[127.23, 117.03, 125.95, 118.32, 124.66, 118.3..."
4,1,L1Ap5_1-1-142.png,Track3,0,364.27,251.51,356.98,269.74,347.26,291.61,Val,332.86,210.92,46.27,66.88,"[355.99, 243.07, 354.7, 244.35, 354.7, 245.64,..."


In [5]:
import numpy as np

def evaluate_instance(gt_lst, outputs):

  instances = len(gt_lst)
  predictions = outputs.shape[0]
  eval = np.ones((instances, predictions)) * -1

  for i in range(instances):
    for j in range(predictions):
      intersection = np.logical_and(gt_lst[i], outputs[j, :, :]).sum()
      union = np.logical_or(gt_lst[i], outputs[j, :, :]).sum()
      eval[i, j] = intersection/union
  return eval

In [6]:
from tqdm import tqdm

def FP_FN_0(output_dict: dict):

  out = 0
  for img in tqdm(output_dict.keys()):
    con_mat_shape = output_dict[img].shape
    if con_mat_shape[0] != con_mat_shape[1]:
      continue
    else:
      tps = output_dict[img] > 0.5
      if (tps.sum(axis = 0) == np.ones(con_mat_shape[0])).all() and (tps.sum(axis = 1) == np.ones(con_mat_shape[0])).all():
        out += 1
  return out

def TP(output_dict: dict):

    out = 0
    for img in tqdm(output_dict.keys()):
      cf_mat = output_dict[img].copy()
      con_mat_shape = cf_mat.shape
      for i in range(con_mat_shape[1]):
        idx = cf_mat[:, i].argmax()
        if cf_mat[idx, i] > 0.5:
          out += 1
          cf_mat[idx, :] = 0
          cf_mat[:, i] = 0
    return out

def FP(output_dict: dict):

  out = 0
  for img in tqdm(output_dict.keys()):
      cf_mat = output_dict[img].copy()
      con_mat_shape = cf_mat.shape
      for i in range(con_mat_shape[1]):
        if (cf_mat[:, i] > 0.5).sum() == 0:
          out += 1
  return out

def TP_0(output_dict: dict):

  out = 0
  for img in tqdm(output_dict.keys()):
      cf_mat = output_dict[img].copy()
      con_mat_shape = cf_mat.shape
      flag = True
      for i in range(con_mat_shape[1]):
        if (cf_mat[:, i] > 0.5).sum() != 0:
          flag = False
      if flag:
        out += 1
  return out

In [7]:
def correct_segmentations(seg):
  corrected_seg = []
  for i in seg[1:-1].split(","):
    corrected_seg.append(float(i))
  return corrected_seg

In [8]:
import matplotlib.pyplot as plt

def plot_img_points_bbox_segcontour(gt_path, labels_df, idx):
  # idx = random.randint(0, labels_df.shape[0])
  img_path = os.path.join(gt_path, labels_df.folder[idx], labels_df.name[idx])
  title = labels_df.name[idx]
  # Read in the image and plot it using matplotlib
  img = mpimg.imread(img_path)
  plt.imshow(img, cmap = "gray")
  colors = ["red", "yellow", "green", "blue", "orange", "pink", "violet", "midnightblue", "tan", "palegreen"]


  image_name = labels_df.name[idx]
  view = labels_df.name == image_name


  points = labels_df.loc[view, ['p1_x', 'p1_y', 'p2_x', 'p2_y','p3_x', 'p3_y']].values
  bbox = labels_df.loc[view, ["bbox_x", "bbox_y", "bbox_width", "bbox_height"]].values.copy()
  segmentation = labels_df.loc[view, "segmentation"].values.copy()
  for j in range(points.shape[0]):
    plt.scatter([points[j,0], points[j,2], points[j,4]], [points[j,1], points[j,3], points[j,5]], color=colors[j], s = 20)
    # plt.scatter(np.array([points[0], points[2], points[4]]), np.array([points[1], points[3], points[5]]), color= "red", s = 2)


    bbox_x = np.array([bbox[j][0], bbox[j][0] + bbox[j][2], bbox[j][0] + bbox[j][2], bbox[j][0], bbox[j][0]])
    bbox_y = np.array([img.shape[0] - bbox[j][1], img.shape[0] - bbox[j][1],
                      img.shape[0] - bbox[j][1]-bbox[j][3],
                      img.shape[0] - bbox[j][1]-bbox[j][3],
                      img.shape[0] - bbox[j][1]])
    # bbox_x = np.array([bbox[0], bbox[0] + bbox[2], bbox[0] + bbox[2], bbox[0], bbox[0]])
    # bbox_y = np.array([img.shape[0] - bbox[1], img.shape[0] - bbox[1],
    #                  img.shape[0] - bbox[1]-bbox[3],
    #                  img.shape[0] - bbox[1]-bbox[3],
    #                  img.shape[0] - bbox[1]])
    plt.plot(bbox_x, bbox_y, color = colors[j], linewidth = 0.2)


    x_segmentation = np.array(correct_segmentations(segmentation[j])[::2])
    y_segmentation = np.array(correct_segmentations(segmentation[j])[1::2])
    plt.plot(x_segmentation, y_segmentation, color = colors[j], linewidth = 0.5)


  plt.title(title)
  plt.axis("off");

  # print(f"Image shape: {img.shape}") # show the shape of the image

  return img

In [9]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_test", {}, Data_cfg["Coco_labels_train_dir"], os.path.join(Data_cfg["cropped_Images_dir"], "Test"))

In [ ]:
# imgs_names = ["L6Jn0_1-1-252.png", "L1Ap5_5-1-9.png", "L1Ap5_1-1-142.png",
#               "L1Ap5_16-1-67.png", "L1Ap5_12-1-79.png", "L1Ap5_15-1-262.png",
#               "L4Jn0_0-1-254.png", "L5Jn0_9-1-48.png"]
# imgs_names = ["L6Jn0_8-1-43.png", "L6Jn0_11-1-210.png", "L1Ap5_7-1-100.png",
#              "L1Ap5_7-1-203.png", "L6Jn0_1-1-39.png", "L5Jn0_7-0-203.png",
#              "L6Jn0_7-1-149.png", "L1Ap5_2-1-168.png", "R1Ap5_29-1-60.png",
#              "L1Ap5_30-1-58.png", "L4Jn0_4-1-96.png"]

# 4

## X101_l1smooth_0.975_0.005_True_8

In [177]:
from detectron2 import model_zoo

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
# cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_tes",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
# cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

cfg.OUTPUT_DIR = "4/X101_l1smooth_0.975_0.005_True_8"

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0002499.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [178]:
from tqdm import tqdm
import cv2

test = df[df.folder == "Test"].copy()
test = test.reset_index().drop(columns = ["index"])
output_dict = {}

for image in tqdm(np.unique(test.name.values)):
  temp = test[test.name == image].copy()
  im_dir = os.path.join(Data_cfg["cropped_Images_dir"], temp.folder.values[0], image)
  im = cv2.imread(im_dir)
  outputs = predictor(im)
  outputs = outputs["instances"].pred_masks.to("cpu").numpy()
  gt_lst = []
  for instance in temp.track:
    gt_dir = os.path.join(Data_cfg["GT_Images_dir"], temp.folder.values[0], temp.name.values[0][:-4], instance + ".png")
    gt = cv2.imread(gt_dir)
    gt = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)
    gt = cv2.resize(gt, (658, 517), interpolation = cv2.INTER_AREA)
    gt = gt.astype("bool")
    gt_lst.append(gt)

  output_dict[image] = evaluate_instance(gt_lst, outputs)

100%|██████████| 98/98 [00:27<00:00,  3.57it/s]


In [179]:
# try:
#     import cPickle as pickle
# except ImportError:  # Python 3.x
#     import pickle
#
# with open(os.path.join(cfg.OUTPUT_DIR, "test_results.p"), 'wb') as fp:
#     pickle.dump(output_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [180]:
# try:
#     import cPickle as pickle
# except ImportError:  # Python 3.x
#     import pickle
#
# with open(os.path.join(cfg.OUTPUT_DIR, "test_results.p"), 'rb') as fp:
#     output_dict = pickle.load(fp)

In [181]:
results_dict = {
    "FP_FN_0": FP_FN_0(output_dict),
    "TP": TP(output_dict),
    "FP": FP(output_dict),
    "TP_0": TP_0(output_dict)
}
results = pd.DataFrame(results_dict, index = ["results"])

100%|██████████| 98/98 [00:00<00:00, 37100.98it/s]


In [182]:
results_dict["Precision"] = round((results.TP/(results.TP+results.FP)).values[0], 4)
results_dict["Recall"] = round((results.TP/286).values[0], 4)
results_dict["Accuracy"] = round((results.FP_FN_0/98).values[0], 4)
results = pd.DataFrame(results_dict, index = ["results"])
results.to_csv(os.path.join(cfg.OUTPUT_DIR, "results.csv"))
results

,FP_FN_0,TP,FP,TP_0,Precision,Recall,Accuracy
results,65,257,23,0,0.9179,0.8986,0.6633


In [183]:
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
from tqdm import tqdm

folder_path = os.path.join(os.getcwd(), cfg.OUTPUT_DIR, "test_images")

# Check if the directory does not exist
if not os.path.exists(folder_path):
    try:
        # Create the directory
        os.makedirs(folder_path)
        print(f"Directory '{folder_path}' was created successfully")
    except OSError as error:
        print(f"Creation of the directory {folder_path} failed due to: {error}")
else:
    print(f"Directory '{folder_path}' already exists")

for im_name in tqdm(np.unique(test.name.values)):
  temp = test[test.name == im_name].copy()
  im_dir = os.path.join(Data_cfg["cropped_Images_dir"], temp.folder.values[0], im_name)
  im = cv2.imread(im_dir)
  outputs = predictor(im)
  v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  save_image = out.get_image()[:, :, ::-1]
  cv2.imwrite(os.path.join(os.getcwd(), cfg.OUTPUT_DIR, "test_images", im_name), save_image)

Directory '/content/drive/MyDrive/Thesis_Organized/Detectron2/Inference_testset/4/X101_l1smooth_0.975_0.005_True_8/test_images' already exists


100%|██████████| 98/98 [00:39<00:00,  2.47it/s]


## X101_l1smooth_0.985_0.001_True_8

In [184]:
from detectron2 import model_zoo

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
# cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_tes",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
# cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

cfg.OUTPUT_DIR = "4/X101_l1smooth_0.985_0.001_True_8"

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0002999.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.825  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [185]:
from tqdm import tqdm
import cv2

test = df[df.folder == "Test"].copy()
test = test.reset_index().drop(columns = ["index"])
output_dict = {}

for image in tqdm(np.unique(test.name.values)):
  temp = test[test.name == image].copy()
  im_dir = os.path.join(Data_cfg["cropped_Images_dir"], temp.folder.values[0], image)
  im = cv2.imread(im_dir)
  outputs = predictor(im)
  outputs = outputs["instances"].pred_masks.to("cpu").numpy()
  gt_lst = []
  for instance in temp.track:
    gt_dir = os.path.join(Data_cfg["GT_Images_dir"], temp.folder.values[0], temp.name.values[0][:-4], instance + ".png")
    gt = cv2.imread(gt_dir)
    gt = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)
    gt = cv2.resize(gt, (658, 517), interpolation = cv2.INTER_AREA)
    gt = gt.astype("bool")
    gt_lst.append(gt)

  output_dict[image] = evaluate_instance(gt_lst, outputs)

100%|██████████| 98/98 [00:27<00:00,  3.52it/s]


In [186]:
# try:
#     import cPickle as pickle
# except ImportError:  # Python 3.x
#     import pickle
#
# with open(os.path.join(cfg.OUTPUT_DIR, "test_results.p"), 'wb') as fp:
#     pickle.dump(output_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [187]:
# try:
#     import cPickle as pickle
# except ImportError:  # Python 3.x
#     import pickle
#
# with open(os.path.join(cfg.OUTPUT_DIR, "test_results.p"), 'rb') as fp:
#     output_dict = pickle.load(fp)

In [188]:
results_dict = {
    "FP_FN_0": FP_FN_0(output_dict),
    "TP": TP(output_dict),
    "FP": FP(output_dict),
    "TP_0": TP_0(output_dict)
}
results = pd.DataFrame(results_dict, index = ["results"])

100%|██████████| 98/98 [00:00<00:00, 41569.76it/s]


In [189]:
results_dict["Precision"] = round((results.TP/(results.TP+results.FP)).values[0], 4)
results_dict["Recall"] = round((results.TP/286).values[0], 4)
results_dict["Accuracy"] = round((results.FP_FN_0/98).values[0], 4)
results = pd.DataFrame(results_dict, index = ["results"])
results.to_csv(os.path.join(cfg.OUTPUT_DIR, "results.csv"))
results

,FP_FN_0,TP,FP,TP_0,Precision,Recall,Accuracy
results,63,253,15,0,0.944,0.8846,0.6429


In [190]:
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
from tqdm import tqdm

folder_path = os.path.join(os.getcwd(), cfg.OUTPUT_DIR, "test_images")

# Check if the directory does not exist
if not os.path.exists(folder_path):
    try:
        # Create the directory
        os.makedirs(folder_path)
        print(f"Directory '{folder_path}' was created successfully")
    except OSError as error:
        print(f"Creation of the directory {folder_path} failed due to: {error}")
else:
    print(f"Directory '{folder_path}' already exists")

for im_name in tqdm(np.unique(test.name.values)):
  temp = test[test.name == im_name].copy()
  im_dir = os.path.join(Data_cfg["cropped_Images_dir"], temp.folder.values[0], im_name)
  im = cv2.imread(im_dir)
  outputs = predictor(im)
  v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  save_image = out.get_image()[:, :, ::-1]
  cv2.imwrite(os.path.join(os.getcwd(), cfg.OUTPUT_DIR, "test_images", im_name), save_image)

Directory '/content/drive/MyDrive/Thesis_Organized/Detectron2/Inference_testset/4/X101_l1smooth_0.985_0.001_True_8/test_images' already exists


100%|██████████| 98/98 [00:39<00:00,  2.46it/s]
